# Optuna untuk Hyperparameter Tuning XGBoost

## Section 1: Penggunaan Simple Optuna

**Apa itu Optuna?**

Optuna adalah library Python yang dirancang khusus untuk melakukan **hyperparameter optimization** secara otomatis. Bayangkan Anda sedang memasak dan ingin menemukan kombinasi bumbu terbaik - Optuna seperti asisten yang secara cerdas mencoba berbagai kombinasi bumbu hingga menemukan rasa yang paling enak.

**Bagaimana Optuna Bekerja?**

1. **Trial**: Optuna menjalankan banyak "trial" atau percobaan. Setiap trial adalah satu kali training model dengan kombinasi hyperparameter yang berbeda.

2. **Sampling Strategy**: Optuna menggunakan algoritma cerdas (seperti TPE - Tree-structured Parzen Estimator) untuk memilih hyperparameter yang akan dicoba selanjutnya. Ini jauh lebih efisien daripada random search atau grid search.

3. **Objective Function**: Kita mendefinisikan fungsi objektif yang mengembalikan metrik yang ingin dioptimalkan (misalnya akurasi atau AUC).

4. **Study**: Optuna mencatat semua trial dalam sebuah "study" dan terus mencari kombinasi hyperparameter terbaik.

**Keuntungan Optuna:**
- Otomatis dan efisien
- Tidak perlu mendefinisikan semua kombinasi hyperparameter di muka
- Bisa berhenti lebih awal jika sudah menemukan hasil yang bagus
- Mudah digunakan dan terintegrasi dengan berbagai ML library

In [1]:
!pip install optuna optuna-integration

In [2]:
# Import library yang diperlukan
import numpy as np
import optuna
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):
    """
    Fungsi objektif untuk Optuna.

    Parameter 'trial' adalah objek yang diberikan Optuna untuk setiap percobaan.
    Trial ini berisi metode untuk 'menyarankan' nilai hyperparameter yang akan dicoba.

    Fungsi ini harus mengembalikan nilai yang ingin kita optimalkan (dalam hal ini akurasi).
    Optuna akan mencoba memaksimalkan nilai ini.
    """

    # Load dataset breast cancer dari scikit-learn
    # Dataset ini cocok untuk binary classification
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)

    # Split data menjadi training dan validation set
    # random_state=42 memastikan split yang konsisten di setiap trial
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25, random_state=42)

    # Konversi data ke format DMatrix yang digunakan XGBoost
    # DMatrix adalah format data khusus XGBoost yang lebih efisien
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    # Definisi hyperparameter untuk XGBoost
    # Optuna akan menyarankan nilai untuk hyperparameter ini di setiap trial
    param = {
        "verbosity": 0,  # Mengurangi output log XGBoost
        "objective": "binary:logistic",  # Untuk binary classification
        "tree_method": "exact",  # Metode untuk membangun tree

        # trial.suggest_categorical: memilih dari daftar kategori
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),

        # trial.suggest_float: memilih nilai float dalam range tertentu
        # log=True berarti sampling dalam skala logaritmik (bagus untuk parameter seperti learning rate)
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),  # L2 regularization
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),   # L1 regularization
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),  # Proporsi sampel yang digunakan
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),  # Proporsi fitur yang digunakan
    }

    # Hyperparameter khusus untuk booster tree-based (gbtree dan dart)
    if param["booster"] in ["gbtree", "dart"]:
        # trial.suggest_int: memilih nilai integer dalam range tertentu
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)  # Kedalaman maksimum tree
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)  # Minimum weight untuk child node
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)  # Learning rate
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)  # Minimum split loss
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    # Hyperparameter tambahan khusus untuk DART booster
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)  # Dropout rate
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)  # Probabilitas skip dropout

    # Training model XGBoost dengan hyperparameter yang disarankan
    bst = xgb.train(param, dtrain)

    # Prediksi pada validation set
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)  # Konversi probabilitas ke label (0 atau 1)

    # Hitung akurasi sebagai metrik yang ingin dioptimalkan
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)

    # Return nilai yang ingin dimaksimalkan oleh Optuna
    return accuracy


# Buat study Optuna
# direction="maximize" berarti kita ingin memaksimalkan nilai yang dikembalikan objective function
print("Memulai optimisasi hyperparameter dengan Optuna...")
study = optuna.create_study(direction="maximize")

# Jalankan optimisasi
# n_trials=100: Optuna akan mencoba 100 kombinasi hyperparameter yang berbeda
# timeout=600: Batas waktu maksimal 600 detik (10 menit)
print("Menjalankan 100 trial optimisasi...")
study.optimize(objective, n_trials=100, timeout=600)

# Tampilkan hasil optimisasi
print(f"\nOptimisasi selesai!")
print(f"Jumlah trial yang diselesaikan: {len(study.trials)}")
print(f"\nTrial terbaik:")
trial = study.best_trial
print(f"  Nilai akurasi terbaik: {trial.value:.4f}")
print(f"  Hyperparameter terbaik:")
for k, v in trial.params.items():
    print(f"    {k}: {v}")

[I 2025-10-18 11:31:22,580] A new study created in memory with name: no-name-44e86163-696b-4f09-bdb8-a5bf5d184de6
[I 2025-10-18 11:31:22,652] Trial 0 finished with value: 0.6223776223776224 and parameters: {'booster': 'dart', 'lambda': 0.035388659886853094, 'alpha': 0.00016117644844843002, 'subsample': 0.9839088055470666, 'colsample_bytree': 0.23548859860088564, 'max_depth': 3, 'min_child_weight': 9, 'eta': 2.2432341476232416e-07, 'gamma': 0.022030879100583056, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.3513013121365335, 'skip_drop': 0.004283410598180803}. Best is trial 0 with value: 0.6223776223776224.
[I 2025-10-18 11:31:22,724] Trial 1 finished with value: 0.6223776223776224 and parameters: {'booster': 'gbtree', 'lambda': 3.999643859800902e-06, 'alpha': 0.033667459979099375, 'subsample': 0.8543730495742541, 'colsample_bytree': 0.9381613166303691, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.000798475353516788, 'gamma': 2.544874

Memulai optimisasi hyperparameter dengan Optuna...
Menjalankan 100 trial optimisasi...


[I 2025-10-18 11:31:22,799] Trial 2 finished with value: 0.6223776223776224 and parameters: {'booster': 'dart', 'lambda': 1.6676400176977503e-06, 'alpha': 0.8284378449563328, 'subsample': 0.49881052168697204, 'colsample_bytree': 0.5126096400805067, 'max_depth': 3, 'min_child_weight': 9, 'eta': 0.00016149585069271982, 'gamma': 1.43556296258011e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 4.6494335215602876e-06, 'skip_drop': 0.16341502399016708}. Best is trial 0 with value: 0.6223776223776224.
[I 2025-10-18 11:31:22,867] Trial 3 finished with value: 0.965034965034965 and parameters: {'booster': 'dart', 'lambda': 0.0007661113092762158, 'alpha': 2.0933140790297765e-08, 'subsample': 0.8338711661741152, 'colsample_bytree': 0.9330769720560379, 'max_depth': 7, 'min_child_weight': 7, 'eta': 0.832824086747103, 'gamma': 6.000395236313489e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 5.6105199


Optimisasi selesai!
Jumlah trial yang diselesaikan: 100

Trial terbaik:
  Nilai akurasi terbaik: 0.9930
  Hyperparameter terbaik:
    booster: dart
    lambda: 1.9263409152582747e-05
    alpha: 8.291469406530223e-08
    subsample: 0.6215172742435517
    colsample_bytree: 0.7176232464865162
    max_depth: 9
    min_child_weight: 6
    eta: 0.38888065386749715
    gamma: 4.5980411015626557e-07
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.0010952875012022532
    skip_drop: 2.6087176377661578e-08


# Section 2: Optuna dengan Pruning

## Apa itu Pruning?

**Pruning** adalah fitur canggih di Optuna yang memungkinkan untuk **menghentikan trial yang tidak menjanjikan lebih awal**. Ini seperti seorang pelatih yang bisa mengenali bahwa seorang atlet tidak akan menang dalam perlombaan, sehingga memutuskan untuk menghentikan partisipasinya dan menghemat energi untuk atlet lain yang lebih berpotensi.

## Bagaimana Pruning Bekerja?

1. **Early Stopping Cerdas**: Selama training model, Optuna memantau performa model secara berkala (misalnya setiap beberapa epoch).

2. **Perbandingan dengan Trial Lain**: Pruner membandingkan performa trial saat ini dengan trial-trial sebelumnya pada titik yang sama dalam proses training.

3. **Keputusan Pruning**: Jika trial saat ini menunjukkan performa yang jauh lebih buruk dibanding trial lain pada titik yang sama, maka trial tersebut dihentikan lebih awal.

4. **Penghematan Waktu**: Dengan menghentikan trial yang tidak menjanjikan, kita bisa menggunakan waktu dan resource untuk mencoba kombinasi hyperparameter lain yang lebih berpotensi.

## Jenis-jenis Pruner di Optuna:

- **MedianPruner**: Menghentikan trial jika performanya di bawah median dari trial-trial sebelumnya
- **PercentilePruner**: Menghentikan trial jika performanya di bawah persentil tertentu
- **SuccessiveHalvingPruner**: Menggunakan algoritma successive halving
- **HyperbandPruner**: Kombinasi dari successive halving dengan multiple brackets

## Keuntungan Pruning:

✅ **Efisiensi Waktu**: Training yang lebih cepat karena trial buruk dihentikan lebih awal  
✅ **Resource Optimization**: Menghemat CPU/GPU untuk trial yang lebih menjanjikan  
✅ **Hasil Lebih Baik**: Dengan waktu yang sama, bisa mencoba lebih banyak kombinasi hyperparameter  
✅ **Otomatis**: Tidak perlu intervention manual untuk menghentikan trial yang buruk

In [3]:
# Import library yang sama seperti sebelumnya
import numpy as np
import optuna
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

def objective(trial):
    """
    Objective function dengan implementasi pruning callback.

    Perbedaan dengan versi sebelumnya:
    1. Menggunakan eval_metric untuk monitoring selama training
    2. Menggunakan XGBoostPruningCallback untuk pruning otomatis
    3. Training dengan evaluasi berkala (evals parameter)
    """

    # Load dan split data (sama seperti sebelumnya)
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25, random_state=42)

    # Konversi ke DMatrix
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    # Hyperparameter configuration
    # eval_metric="auc" penting untuk pruning - ini yang akan dimonitor
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",  # Metrik yang akan dimonitor untuk pruning
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    # Hyperparameter untuk tree-based boosters
    if param["booster"] in ("gbtree", "dart"):
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    # Hyperparameter tambahan untuk DART
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # KUNCI PRUNING: Buat XGBoostPruningCallback
    # Callback ini akan memantau "validation-auc" selama training
    # Jika performa buruk dibanding trial lain, akan menghentikan training lebih awal
    pruning_cb = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")

    # Training dengan monitoring dan callback pruning
    # evals: mendefinisikan dataset untuk evaluasi (validation set)
    # callbacks: list callback yang akan dijalankan, termasuk pruning callback
    bst = xgb.train(
        param,
        dtrain,
        evals=[(dvalid, "validation")],  # Dataset untuk evaluasi dan monitoring
        callbacks=[pruning_cb],  # Callback untuk pruning otomatis
        verbose_eval=False  # Mengurangi output selama training
    )

    # Prediksi dan evaluasi final (sama seperti sebelumnya)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy


# Buat study dengan pruner
# MedianPruner: menghentikan trial jika performanya di bawah median
# n_warmup_steps=5: tidak melakukan pruning pada 5 step pertama (biarkan model "warming up")
print("Memulai optimisasi dengan fitur pruning...")
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    direction="maximize",
)

# Jalankan optimisasi (tanpa timeout kali ini, hanya berdasarkan jumlah trial)
print("Menjalankan optimisasi dengan pruning otomatis...")
print("Trial yang tidak menjanjikan akan dihentikan lebih awal...")
study.optimize(objective, n_trials=100)

# Tampilkan hasil
best = study.best_trial
print(f"\nOptimisasi dengan pruning selesai!")
print(f"Jumlah trial yang diselesaikan: {len(study.trials)}")

# Hitung berapa trial yang di-prune
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
print(f"Jumlah trial yang di-prune (dihentikan lebih awal): {len(pruned_trials)}")
print(f"Efisiensi pruning: {len(pruned_trials)/len(study.trials)*100:.1f}% trial dihentikan lebih awal")

print(f"\nTrial terbaik (dengan pruning):")
print(f"  Nilai akurasi terbaik: {best.value:.4f}")
print(f"  Hyperparameter terbaik:")
for k, v in best.params.items():
    print(f"    {k}: {v}")

[I 2025-10-18 11:32:30,412] A new study created in memory with name: no-name-5b63fd9d-0788-4430-8641-4091f9cfd94a
[I 2025-10-18 11:32:30,469] Trial 0 finished with value: 0.6223776223776224 and parameters: {'booster': 'gbtree', 'lambda': 0.019576700477762526, 'alpha': 1.4166871828443767e-06, 'max_depth': 2, 'eta': 0.0001239278795392276, 'gamma': 2.371421247731785e-08, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.6223776223776224.
[I 2025-10-18 11:32:30,530] Trial 1 finished with value: 0.916083916083916 and parameters: {'booster': 'gblinear', 'lambda': 2.580193976331876e-07, 'alpha': 0.08923127833946408}. Best is trial 1 with value: 0.916083916083916.
[I 2025-10-18 11:32:30,601] Trial 2 finished with value: 0.9440559440559441 and parameters: {'booster': 'gblinear', 'lambda': 2.178190342557355e-07, 'alpha': 0.0025731853569676212}. Best is trial 2 with value: 0.9440559440559441.


Memulai optimisasi dengan fitur pruning...
Menjalankan optimisasi dengan pruning otomatis...
Trial yang tidak menjanjikan akan dihentikan lebih awal...


[I 2025-10-18 11:32:30,694] Trial 3 finished with value: 0.9440559440559441 and parameters: {'booster': 'gblinear', 'lambda': 4.44735713116522e-05, 'alpha': 0.003278062361505308}. Best is trial 2 with value: 0.9440559440559441.
[I 2025-10-18 11:32:30,758] Trial 4 finished with value: 0.916083916083916 and parameters: {'booster': 'gblinear', 'lambda': 0.13621037159920535, 'alpha': 0.0008135774971468205}. Best is trial 2 with value: 0.9440559440559441.
[I 2025-10-18 11:32:30,868] Trial 5 pruned. Trial was pruned at iteration 5.
[I 2025-10-18 11:32:30,953] Trial 6 pruned. Trial was pruned at iteration 5.
[I 2025-10-18 11:32:31,043] Trial 7 pruned. Trial was pruned at iteration 5.
[I 2025-10-18 11:32:31,224] Trial 8 finished with value: 0.972027972027972 and parameters: {'booster': 'dart', 'lambda': 0.04792973808017873, 'alpha': 0.004346312644823983, 'max_depth': 4, 'eta': 0.2960550058222188, 'gamma': 4.750066312132874e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_


Optimisasi dengan pruning selesai!
Jumlah trial yang diselesaikan: 100
Jumlah trial yang di-prune (dihentikan lebih awal): 73
Efisiensi pruning: 73.0% trial dihentikan lebih awal

Trial terbaik (dengan pruning):
  Nilai akurasi terbaik: 0.9720
  Hyperparameter terbaik:
    booster: dart
    lambda: 0.04792973808017873
    alpha: 0.004346312644823983
    max_depth: 4
    eta: 0.2960550058222188
    gamma: 4.750066312132874e-05
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: forest
    rate_drop: 3.905775037103634e-05
    skip_drop: 1.477545634971322e-06


Dengan pruning, kita bisa menghemat waktu karena trial yang buruk dihentikan lebih awal, sehingga bisa fokus pada hyperparameter yang menjanjikan!